In [52]:
import numpy as np
from scipy.stats import truncnorm
import math
import data_extraction_script_module2

C:\Users\Maxime Calloix\Documents\ENSAE\3A\Hidden Markov Chain\Project\Code\data_extraction_script_module2.py:123: UserWarning: WARNING : There is more than 365 days or less than 365 days in dataframe
  warnings.warn("WARNING : There is more than 365 days or less than 365 days in dataframe")


Historique_consommation_INST_2012/Historique_consommation_INST_2012.xls
Number of rows 366


In [3]:
#theta_generation : this function generates a theta according to the laws described in the article
def theta_generation(number_of_daytypes):
    sigma2_s = 1 / (np.random.gamma(size = 1, shape = 0.01, scale = 100)[0]) 
    sigma2_g = 1 / (np.random.gamma(size = 1, shape = 0.01, scale = 100)[0]) 
    u_heat = np.random.normal(size=1, loc = 14, scale = 1)[0] #loc = mean / scale = standard deviation /!\
    kappa = (np.random.dirichlet(size = 1, alpha = [1]*number_of_daytypes)[0])*number_of_daytypes
    sigma2 = 1 / (np.random.gamma(size = 1, shape = 0.01, scale = 100)[0]) 
    return(sigma2_s,sigma2_g,u_heat,kappa,sigma2)

In [54]:
#x_season_generation : simulation of the x_season according to the law described in the article
def x_season_generation(n,electricity_data,kappa, sigma2_s):
    #Initialization
    x = np.zeros(n)
    nu = truncnorm.rvs(a = 0, b = math.inf, loc= 0, scale = sigma2_s, size=1)[0]
    sigma2_s_n = nu
    error = truncnorm.rvs(a = 0, b = math.inf, loc= 0, scale = sigma2_s_n, size=1)[0]
    s_n=error
    x[0] = s_n * kappa[electricity_data.df.Day_type[0]] # on multiplie s_n par la valeur de kappa au jour n
    for i in range(1,n,1):
        nu = truncnorm.rvs(a = -sigma2_s_n / sigma2_s , b = math.inf, loc= 0, scale = sigma2_s, size=1)[0]
        sigma2_s_n += nu
        error = truncnorm.rvs(a = -s_n / sigma2_s_n , b = math.inf, loc= 0, scale = sigma2_s_n, size=1)[0]
        s_n += error
        x[i] = s_n * kappa[electricity_data.df.Day_type[i]] # on multiplie s_n par la valeur de kappa au jour n  
    return(x)

In [55]:
#x_heat_generation : simulation of the x_heat according to the law described in the article
def x_heat_generation(n,temperature_data,u_heat, sigma2_g, hour):
    #Initialization
    x = np.zeros(n)
    nu = truncnorm.rvs(a = 0, b = math.inf, loc= 0, scale = sigma2_g, size=1)[0]
    sigma2_g_n = nu
    error = truncnorm.rvs(a = -math.inf, b = 0, loc= 0, scale = sigma2_g_n, size=1)[0]
    g_n=error
    x[0] = g_n * min(temperature_data.df.iloc[0,hour]-u_heat,0) # on multiplie g_n par la différence de la température
    for i in range(1,n,1):
        nu = truncnorm.rvs(a = -sigma2_g_n / sigma2_g , b = math.inf, loc= 0, scale = sigma2_g, size=1)[0]
        sigma2_g_n += nu
        error = truncnorm.rvs(a = - math.inf , b = -g_n / sigma2_g_n , loc= 0, scale = sigma2_g_n, size=1)[0]
        g_n += error
        x[i] = g_n * min(temperature_data.df.iloc[i, hour]-u_heat,0) # on multiplie s_n par la différence de la température 
    return(x)

In [50]:
#electricity_simulation : function to be called in order to retrieve the electricity simulation
# We compute separately v (the main error), x_season and x_heat and we sum them at the end
#Parameters are :
# - an instance of the class for the temperature
# - an instance of the class for the electricity(in order to retrieve the day-type)
# - the vector theta from theta_generation
# - the index of the hour
def electricity_simulation(temperature_data,electricity_data,theta,hour):
    sigma2_s,sigma2_g,u_heat,kappa,sigma2 = theta
    n = len(temperature_data.df)
    v = np.random.normal(size = n, loc = 0, scale = sigma2) #loc = mean / scale = standard deviation /!\
    return(v + x_season_generation(n,electricity_data,kappa, sigma2_s) + x_heat_generation(n,temperature_data,u_heat, sigma2_g, hour ) )

Example of the code used

In [ ]:
#Loading of the data
temperature_data = data_extraction_script_module2.Temperatures()
temperature_data.get_temperatures('C:/Users/Maxime Calloix/Documents/ENSAE/3A/Hidden Markov Chain/Project/Donnees temperature/')
electricity_data = data_extraction_script_module2.Electricity()
electricity_data.get_electricity_data("C:/Users/Maxime Calloix/Documents/ENSAE/3A/Hidden Markov Chain/Project/Donnees electricite/")

In [67]:
number_of_daytypes = len(set(electricity_data.df.Day_type))
theta = theta_generation(number_of_daytypes)
print(theta)#sigma2_s, sigma2_g, u_heat, kappa, sigma2

(11209532.315395907, 9.6901990981059529e+40, 14.249459396388492, array([ 3.04722535,  0.13057057,  0.29475074,  0.79769772,  0.37695658,
        1.02041211,  1.26916969,  1.06321723]), 43078.88512142151)


In [68]:
simulation = electricity_simulation(temperature_data,electricity_data,theta,0)#We use the hour 00:00

In [69]:
print(simulation)

[  4.88873400e+40   3.32586732e+41   1.16090225e+42 ...,   1.66916756e+45
   2.46831582e+45   1.71334218e+45]
